In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/T5 && zip -r T5.zip .


  adding: train.csv (deflated 70%)
  adding: test.csv (deflated 70%)


In [ ]:
!ls -lh "/content/drive/My Drive/StackOverflow_Data/new/tag-list_cleaned_2.csv"


-rw------- 1 root root 12G Oct 16 04:34 '/content/drive/My Drive/StackOverflow_Data/new/tag-list_cleaned_2.csv'


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/StackOverflow_Data/new/tag-list_cleaned_2.csv'

# Membaca file CSV
df = pd.read_csv(file_path, nrows=10000)
print(df.head())

         Id             CreationDate  \
0  59551209  2020-01-01T09:10:05.390   
1  59551210  2020-01-01T09:10:06.453   
2  59551214  2020-01-01T09:11:01.173   
3  59551215  2020-01-01T09:11:05.170   
4  59551216  2020-01-01T09:11:15.690   

                                               Title  \
0  Android in App Purchase - Service connection i...   
1  parse key=val list where space has dual meanin...   
2               Is there any Audio Testing Hardware?   
3             Multiple vim regex command in one line   
4     C# unable to return json object of large data?   

                                                Body  \
0  <p>I'm constantly receiveing -1 response code ...   
1  <p>I'm looking for extracting the environment ...   
2  <p>We want to test sound characteristics of le...   
3  <p>I want to run ,multiple regex commands in p...   
4  <p>Using C# MVC ASP.NET:</p>\n\n<p>hello, im t...   

                                                Tags  \
0                          |a

In [ ]:
import pandas as pd

FILE_PATH = "/content/drive/MyDrive/StackOverflow_Data/new/tag-list_cleaned_2.csv"

df = pd.read_csv(FILE_PATH, nrows=5)   # hanya 5 baris, AMAN
print(df.columns.tolist())


['Id', 'CreationDate', 'Title', 'Body', 'Tags', 'tag_list']


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

FILE_PATH = "/content/drive/MyDrive/StackOverflow_Data/new/tag-list_cleaned_2.csv"

CHUNK_SIZE = 50_000
TARGET_SAMPLES = 200_000   # boleh 100k kalau mau lebih ringan

print("Load CSV dengan chunking...")

chunks = []
total = 0

for chunk in pd.read_csv(FILE_PATH, chunksize=CHUNK_SIZE):
    # pastikan kolom penting ada
    required_cols = {"Title", "Body", "tag_list"}
    if not required_cols.issubset(chunk.columns):
        raise ValueError(f"Kolom wajib tidak ditemukan: {required_cols}")

    # gabungkan Title + Body → text
    chunk["text"] = (
        chunk["Title"].astype(str) + " " + chunk["Body"].astype(str)
    )

    # cleaning ringan
    chunk = chunk.dropna(subset=["text", "tag_list"])
    chunk = chunk[chunk["tag_list"] != "[]"]

    need = TARGET_SAMPLES - total
    if need <= 0:
        break

    chunk = chunk.sample(min(len(chunk), need), random_state=42)

    chunks.append(chunk[["text", "tag_list"]])
    total += len(chunk)

    print(f"Terkumpul: {total}")

    if total >= TARGET_SAMPLES:
        break

df = pd.concat(chunks, ignore_index=True)

print("Data siap untuk split")
print("Total data:", df.shape)

# =====================================================
# SPLIT 90 / 10
# =====================================================

train_df, test_df = train_test_split(
    df,
    test_size=0.1,
    random_state=42
)

train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

print("Split selesai")
print("Train:", train_df.shape)
print("Test :", test_df.shape)

# sanity check
print("\nContoh TRAIN:")
print(train_df.loc[0])

print("\nContoh TEST:")
print(test_df.loc[0])


Load CSV dengan chunking...
Terkumpul: 50000
Terkumpul: 100000
Terkumpul: 150000
Terkumpul: 200000
Data siap untuk split
Total data: (200000, 2)
Split selesai
Train: (180000, 2)
Test : (20000, 2)

Contoh TRAIN:
text        Unexpected character '#' <p>I'm using webpack ...
tag_list    ['javascript', 'npm', 'webpack', 'ecmascript-n...
Name: 0, dtype: object

Contoh TEST:
text        Simple scrollable list in ASP.NET Core MVC <p>...
tag_list                                 ['asp.net-core-mvc']
Name: 0, dtype: object


In [ ]:
print(train_df.head())
print(train_df.columns)


                                                text  \
0  Unexpected character '#' <p>I'm using webpack ...   
1  azure web jobs swap in secondary slot in app s...   
2  What is the difference between this.value and ...   
3  I am getting unwanted comma in my markup when ...   
4  Node JS await call returns undefined <p>The pr...   

                                            tag_list  
0  ['javascript', 'npm', 'webpack', 'ecmascript-n...  
1  ['azure', 'azure-web-app-service', 'azure-webj...  
2                           ['javascript', 'jquery']  
3                   ['javascript', 'arrays', 'json']  
4  ['javascript', 'node.js', 'express', 'async-aw...  
Index(['text', 'tag_list'], dtype='object')


In [ ]:
import os

SAVE_DIR = "/content/drive/MyDrive/T5"
os.makedirs(SAVE_DIR, exist_ok=True)

TRAIN_PATH = os.path.join(SAVE_DIR, "train.csv")
TEST_PATH  = os.path.join(SAVE_DIR, "test.csv")

print("Menyimpan train_df")
train_df.to_csv(TRAIN_PATH, index=False)

print("Menyimpan test_df")
test_df.to_csv(TEST_PATH, index=False)

print("Selesai menyimpan")
print("Train:", TRAIN_PATH)
print("Test :", TEST_PATH)


Menyimpan train_df
Menyimpan test_df
Selesai menyimpan
Train: /content/drive/MyDrive/T5/train.csv
Test : /content/drive/MyDrive/T5/test.csv


In [ ]:
pd.read_csv(TRAIN_PATH, nrows=3)


,text,tag_list
0,Unexpected character '#' <p>I'm using webpack ...,"['javascript', 'npm', 'webpack', 'ecmascript-n..."
1,azure web jobs swap in secondary slot in app s...,"['azure', 'azure-web-app-service', 'azure-webj..."
2,What is the difference between this.value and ...,"['javascript', 'jquery']"


#Distribusi Label

In [ ]:
from collections import Counter
import ast

def count_labels(df):
    counter = Counter()
    for x in df["tag_list"]:
        tags = ast.literal_eval(x)  # aman untuk string list
        counter.update(tags)
    return counter

train_counts = count_labels(train_df)
test_counts  = count_labels(test_df)

print("Top 15 label TRAIN:")
for k, v in train_counts.most_common(15):
    print(f"{k:20s} {v}")

print("\nTop 15 label TEST:")
for k, v in test_counts.most_common(15):
    print(f"{k:20s} {v}")


Top 15 label TRAIN:
python               25707
javascript           18724
java                 10869
c#                   8704
reactjs              8620
html                 7423
android              6771
r                    6091
php                  5406
c++                  5276
css                  5248
node.js              5066
python-3.x           4811
sql                  4682
pandas               4402

Top 15 label TEST:
python               2900
javascript           2159
java                 1153
c#                   982
reactjs              922
html                 825
android              716
php                  623
c++                  620
r                    606
css                  588
sql                  545
node.js              543
python-3.x           508
flutter              506


In [ ]:
print("Jumlah data:")
print("Train:", len(train_df))
print("Test :", len(test_df))

for label in list(train_counts.keys())[:10]:
    train_n = train_counts[label]
    test_n  = test_counts[label]
    ratio = test_n / max(train_n, 1)
    print(f"{label:20s} test/train ratio: {ratio:.3f}")



Jumlah data:
Train: 180000
Test : 20000
javascript           test/train ratio: 0.115
npm                  test/train ratio: 0.095
webpack              test/train ratio: 0.109
ecmascript-next      test/train ratio: 0.000
azure                test/train ratio: 0.107
azure-web-app-service test/train ratio: 0.116
azure-webjobs        test/train ratio: 0.000
jquery               test/train ratio: 0.127
arrays               test/train ratio: 0.111
json                 test/train ratio: 0.100


In [ ]:
rare_labels = [k for k, v in train_counts.items() if v < 50]

print(f"Jumlah label langka (<50 di train): {len(rare_labels)}")
print("Contoh label langka:", rare_labels[:10])


Jumlah label langka (<50 di train): 23713
Contoh label langka: ['ecmascript-next', 'azure-webjobs', 'undefined', 'curve-fitting', 'google-bucket', 'move', 'emv', 'jsoup', 'python-collections', 'azure-container-apps']


#Training

In [ ]:
!pip install -q transformers accelerate sentencepiece evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
import ast
import torch
import numpy as np
from torch.utils.data import Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from sklearn.metrics import precision_score, recall_score, f1_score

class T5MultiLabelDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len=512, max_target_len=128):
        self.texts = df["text"].tolist()
        self.targets = df["tag_list"].tolist()
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        input_text = "classify: " + self.texts[idx]
        labels = ", ".join(ast.literal_eval(self.targets[idx]))

        enc = self.tokenizer(
            input_text,
            max_length=self.max_input_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        dec = self.tokenizer(
            labels,
            max_length=self.max_target_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        label_ids = dec["input_ids"].squeeze()
        label_ids[label_ids == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": label_ids
        }

MODEL_NAME = "t5-small"   # aman di Colab
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Device:", device)

train_dataset = T5MultiLabelDataset(train_df, tokenizer)
eval_dataset  = T5MultiLabelDataset(test_df, tokenizer)

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    preds_text = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels_text = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ambil semua label unik dari data (dinamis)
    all_labels = set()
    for t in train_df["tag_list"]:
        all_labels.update(ast.literal_eval(t))
    all_labels = sorted(list(all_labels))

    def to_binary(tags):
        return [1 if l in tags else 0 for l in all_labels]

    y_true, y_pred = [], []
    for t, p in zip(labels_text, preds_text):
        y_true.append(to_binary([x.strip() for x in t.split(",") if x.strip()]))
        y_pred.append(to_binary([x.strip() for x in p.split(",") if x.strip()]))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    return {
        "micro_precision": precision_score(y_true, y_pred, average="micro", zero_division=0),
        "micro_recall":    recall_score(y_true, y_pred, average="micro", zero_division=0),
        "micro_f1":        f1_score(y_true, y_pred, average="micro", zero_division=0),
    }

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/t5_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,   # aman
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,   # efektif batch 16
    num_train_epochs=3,
    fp16=True if device.type == "cuda" else False,
    predict_with_generate=True,
    generation_max_length=128,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    report_to="none",
    logging_steps=200,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



Device: cuda


/tmp/ipython-input-429390610.py:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


In [ ]:
print("\n=== Final Evaluation ===")
results = trainer.evaluate()
print(results)

SAVE_DIR = "/content/drive/MyDrive/t5_results/best_model"
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("Model disimpan di:", SAVE_DIR)
